# Creating Semivariograms

An empirical semivariogram is a tool used to describe spatial dependence as a function of separation distance (Ribeiro et al., 2003). Semivariance (the individual values that make up a semivariogram) can be used as a measure of texture in an image (Le´vesque et al., 2009). I used this to measure how rough or smooth the terrain in an area was. To compute the semivariogram each point is compared to every other point. It is common to instead average this value for pairs of points that fall within certain distance intervals or lags. Semivarinace can be expressed by the following equation.

$$\Large
g(h) = \frac{1}{2N_h}\sum^{N}_{i=1}(z_i - z_{i+h})^2
$$

Where $N_h$ is the number of paired elevation values separated by lag $h$, and $z_i$ and $z_{i+h}$ are the elevation values separated by lag $h$. Higher values of $g$ represent higher variance in elevation. A past study by Wu et al. (2009) demonstrated the success of using fifteen lags for land use classification and that number has been used here. For each of the areas in the data set lagged values were computed at lags 0-14 at 40-foot intervals.


## References
- Ribeiro Jr., P. J., Christensen O. F., Diggle, P. J., (2003). geoR and geoRglm: Software for Model-Based Geostatistics. Proceedings of the 3rd International Workshop on Distributed Statistical Computing. https://www.r-project.org/conferences/DSC-2003/Proceedings/RibeiroEtAl.pdf
- Lévesque, J., & King, D. J. (n.d.). Airborne Digital Camera Image Semivariance for Evaluation of Forest Structural Damage at an Acid Mine Site. Remote Sensing of Environment, 68(2), 112–124. https://doi.org/10.1016/S0034-4257(98)00104-7
- Wu, S.-S., Qiu, X., Usery, E. L., & Wang, L. (n.d.). Using Geometrical, Textural, and Contextual Information of Land Parcels for Classification of Detailed Urban Land Use. Annals of the Association of American Geographers, 99(1), 76–98. https://doi.org/10.1080/00045600802459028

# Setup

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

from sklearn.preprocessing import StandardScaler

import skgstat as skg

FILE_PATH = "G:/UNCG_Capstone/"
# File Path Merged Dataset
CLEANED_DATA = FILE_PATH + "Output/Cleaned_Data/"
PICKLE_PATH = FILE_PATH + "Output/Raster_Pickles/"
TRAIN_PATH = FILE_PATH + "Output/Training_Data/"
SEED = 42
NAN = -9999
MAX_SAMPLE = 11255

rng = np.random.default_rng(SEED)

## Functions

In [21]:
def get_coords_values(arr, missing_val = NAN, scale = 20, down_sample_thresh = None, rng = None):
    """Generates a list of x and y coordinates for a 2D array of values and
    a second 1D array containg the values at the x and y coordinates.

    Note this assumes that the data in arr represnts values at regular intervals
    in the case of NC DEM data this means that each point represnts a value 20ft apart

    By multipying by the scale the coordinates returned are in the same scale as the 
    DEM data
    
    ## Parameters
    - arr: array to divide up
    - _val: value in data that means it should be disgarded
    - scale: amount to scale the coordinates by
    - down_sample_thresh: Max number of elvation values to allow. If None, there is no threshold
    - rng: numpy Generator object for sampling
    
    ## Returns
    - An 2D array of xy coordinates
    - A 1D array of elevation values at those coordinates"""

    if scale < 1:
        raise(ValueError("Scale must be an value >= 1"))
    if down_sample_thresh != None and type(rng) != type(np.random.default_rng()):
        raise(ValueError("If a size threshold is specified, an RNG must be passed"))

    coords = np.array([xy for xy in np.ndindex(arr.shape)])
    coords = coords * scale
    values = arr.flatten()

    # Mask out missing values
    if missing_val != None:
        mask = values != missing_val
        coords = coords[mask]
        values = values[mask]

    # If a size threshold has been specified
    if down_sample_thresh != None and values.shape[0] > down_sample_thresh:
        samples = rng.choice(a = values.shape[0], # Sample from all possible values in array
                             size = down_sample_thresh, # Values down to max size
                             replace = False, # No replacement
                             shuffle = False) # Don't shuffle for better speed
        return coords[samples, :], values[samples]
    # No threshold, just return the values
    else:
        return coords, values

def load_raster( id, county, df, fp = PICKLE_PATH):
    """Loads up a raster for a given area. Each area is uniquely defined by it's
    id and county values.
    
    ## Parameters
    - id: id of area load raster data of
    - county: county of area to load raster data of
    - fp: file path to follow for pickled raster arrays"""
    
    land_use =  df.query('id == @id and County == @county')["land_use"].values[0]

    return np.load(fp + f"{land_use}/{county.lower().replace(' ', '_')}_{id}.pickle", allow_pickle = True)


def compute_semivariance(df, standardize = True):
    """For each area in the data set 15 lags are computed. The elevation values may be standardized or
    left as is prior to computing the semivariance. Note that in the dataframe that is returned the min,
    max, mean, and standard deviation are taken from the pre-scaled data regardless of parameters.
    
    ## Parameters
    - df: Dataframe of areas to iterate through
    - standardize: If True the elevation values will have a standard scaling
    
    ## Returns
    A dataframe with 15 lagged values of semivariance, id, enclosing county, mean, min, max, and 
    standard deviation of elevation"""

    # Standard Scaler for each area
    scaler = StandardScaler()

    # Elements to build new dataframe
    ids = []
    counties = []
    lags = []
    # Some descriptive stats of un transformed data
    min = []
    max = []
    mean = []
    sd = []

    for row in df.itertuples():
        id = row[1]
        county = row[2]
        coords, values = get_coords_values(load_raster(id, county, df),
                                        down_sample_thresh = MAX_SAMPLE,
                                        rng = rng)
        
        # Add min and max before scaling
        curr_min = np.min(values)
        curr_max = np.max(values)
        curr_mean = np.mean(values)
        curr_std = np.std(values)
        
        if standardize == True:
            # Apply standardization
            # Note that scaler expects data like [[1], [2], ...] so that is what reshape(-1,1) is for
            values = scaler.fit_transform(values.reshape(-1,1)).flatten()
            
        # Compute Semivariance
        try:
            vario = skg.Variogram(coords, values, maxlag = 600, n_lags=15, normalize=False)
            # Add other stats from scaler
            mean.append(curr_mean)
            sd.append(curr_std)
            min.append(curr_min)
            max.append(curr_max)
            # Add id, county
            ids.append(id)
            counties.append(county)
            # Add lags
            lags.append(vario.experimental)
        except ValueError:
            print(f"ID: {id}, County: {county}")


    # Create np array of lags, will need slice notation
    lags_np = np.array(lags)
    # Create columns for lags
    lag_cols = {f"L{i}" : lags_np[:,i] for i in range(lags_np.shape[1])}
    # Add remaining columns
    lag_cols["id"] = ids
    lag_cols["County"] = counties
    lag_cols["mean"] = np.array(mean).flatten()
    lag_cols["min"] = min
    lag_cols["max"] = max
    lag_cols["sd"] = np.array(sd).flatten()

    return pd.DataFrame( data = lag_cols)

## Data

In [5]:
tp = gpd.read_file(CLEANED_DATA + "trails_parks_merged.shp")
tp.head()

,id,County,area,land_use,geometry
0,trail_row-0,Clay,6.739168e+05,MTB,"POLYGON ((573816.873 491670.935, 573807.105 49..."
1,trail_row-1,Transylvania,3.175570e+06,MTB,"POLYGON ((827189.766 503455.791, 827179.954 50..."
2,trail_row-2,Jackson,4.483349e+05,MTB,"POLYGON ((804142.197 533906.373, 802699.519 53..."
3,trail_row-3,Cherokee,2.175227e+06,MTB,"POLYGON ((502163.436 541197.184, 502153.625 54..."
4,trail_row-4,Transylvania,3.743272e+05,MTB,"POLYGON ((874128.357 564348.367, 874118.580 56..."


# Computing Semivariance

In [22]:
trail_train_std = compute_semivariance(tp)

c:\Users\HoLt\AppData\Local\Programs\Python\Python39\lib\site-packages\skgstat\Variogram.py:2135: RuntimeWarning: All-NaN slice encountered
  bounds = [np.nanmax(x), np.nanmax(y)]


ID: MA_ID-214, County: Currituck


c:\Users\HoLt\AppData\Local\Programs\Python\Python39\lib\site-packages\skgstat\Variogram.py:2135: RuntimeWarning: All-NaN slice encountered
  bounds = [np.nanmax(x), np.nanmax(y)]


ID: MA_ID-730, County: Brunswick


c:\Users\HoLt\AppData\Local\Programs\Python\Python39\lib\site-packages\skgstat\Variogram.py:2135: RuntimeWarning: All-NaN slice encountered
  bounds = [np.nanmax(x), np.nanmax(y)]


ID: MA_ID-65, County: Carteret


In [27]:
trail_train_non = compute_semivariance(tp, standardize = False)

c:\Users\HoLt\AppData\Local\Programs\Python\Python39\lib\site-packages\skgstat\Variogram.py:2135: RuntimeWarning: All-NaN slice encountered
  bounds = [np.nanmax(x), np.nanmax(y)]


ID: MA_ID-214, County: Currituck


c:\Users\HoLt\AppData\Local\Programs\Python\Python39\lib\site-packages\skgstat\Variogram.py:2135: RuntimeWarning: All-NaN slice encountered
  bounds = [np.nanmax(x), np.nanmax(y)]


ID: MA_ID-730, County: Brunswick


c:\Users\HoLt\AppData\Local\Programs\Python\Python39\lib\site-packages\skgstat\Variogram.py:2135: RuntimeWarning: All-NaN slice encountered
  bounds = [np.nanmax(x), np.nanmax(y)]


ID: MA_ID-65, County: Carteret


Some areas located on the coast have issues with missing values, those will be excluded.

In [24]:
trail_train_std.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,...,L11,L12,L13,L14,id,County,mean,min,max,sd
0,0.002631,0.014318,0.039264,0.073553,0.115805,0.164162,0.221299,0.282525,0.345773,0.412199,...,0.544628,0.609530,0.672985,0.733951,trail_row-0,Clay,2065.064941,1921.665039,2253.596924,74.154350
1,0.000895,0.004420,0.011097,0.019099,0.028026,0.037377,0.047165,0.056811,0.065633,0.074424,...,0.091492,0.099640,0.107456,0.115551,trail_row-1,Transylvania,1840.440674,1296.604004,2419.321045,246.355194
2,0.002920,0.015043,0.039445,0.070975,0.107783,0.147908,0.192498,0.238187,0.283724,0.329426,...,0.417017,0.457565,0.496765,0.533538,trail_row-2,Jackson,3876.413574,3746.731934,4019.413086,68.011208
3,0.001340,0.007040,0.018516,0.033397,0.050742,0.069229,0.089308,0.110239,0.130165,0.150693,...,0.190756,0.209441,0.228287,0.246969,trail_row-3,Cherokee,2178.797607,1816.668945,2636.106934,153.023788
4,0.000714,0.003781,0.010195,0.018812,0.029342,0.041413,0.055688,0.071325,0.088064,0.106352,...,0.147102,0.169580,0.193774,0.219025,trail_row-4,Transylvania,3106.330811,2736.946045,3540.008057,189.544449


In [29]:
trail_train_non.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,...,L11,L12,L13,L14,id,County,mean,min,max,sd
0,14.284453,78.048255,214.044008,401.553350,633.537876,902.411435,1214.550193,1551.274676,1899.560177,2264.530874,...,3008.271058,3373.991078,3733.550237,4073.046456,trail_row-0,Clay,2064.956055,1921.665039,2253.596924,74.293549
1,52.178487,259.073510,656.735777,1142.762499,1678.528227,2232.904809,2813.929316,3360.949123,3918.399919,4473.655994,...,5507.390625,5962.144291,6485.574135,6976.281599,trail_row-1,Transylvania,1839.400391,1297.206055,2401.622070,249.978485
2,13.464655,69.031734,181.154628,326.022906,494.790087,678.040284,881.638831,1090.591090,1297.681058,1507.678103,...,1910.240417,2097.773408,2278.825026,2448.373930,trail_row-2,Jackson,3876.554688,3746.731934,4019.413086,67.853653
3,31.684842,164.266855,433.853905,779.029872,1187.148589,1613.924531,2078.551847,2544.428638,2999.254178,3465.940527,...,4327.674565,4760.901894,5201.685819,5630.645195,trail_row-3,Cherokee,2177.131592,1816.119995,2631.754883,152.515549
4,25.650649,135.831226,366.281653,675.857535,1054.183155,1487.807322,2000.677974,2562.515418,3163.802675,3821.072538,...,5284.304517,6092.512543,6962.244423,7869.206835,trail_row-4,Transylvania,3106.330811,2736.946045,3540.008057,189.544449


# Exporting Training Data

## Standardized Elevation

In [26]:
trail_train_std.to_csv(TRAIN_PATH + "training_std.csv", index = False)

## Untransformed Elevation

In [30]:
trail_train_non.to_csv(TRAIN_PATH + "training_non.csv", index = False)